
# 서울 공원·산 블로그 데이터 — 문장 기반 필터링 + 딥러닝 감성분석 (ipynb)
**핵심 아이디어**  
1) 문장 단위로 분리 → 2) 공원/산 관련 문장만 선별(고정 엔티티 매칭, 선택: 임베딩 유사도) →  
3) 해당 문장만 딥러닝 모델에 입력(Transformers) → 4) 엔티티별 긍/부정 집계

**토큰화 전략 요약**
- **문장 토큰화**: `kss`로 한국어 문장 분리(권장).  
- **클린업 전처리**: URL/이모지/HTML/반복문자/제휴문구 제거.  
- **서브워드 토큰화**: 감성모델의 **동일 토크나이저**(예: KcELECTRA/KoBERT tokenizer) 사용.  
- (선택) **형태소 토큰화**는 키워드 분석 보조용으로만 사용.


In [1]:

import re
import os
import pandas as pd
from typing import List

# ===== 경로 설정 =====
INPUT_PATH  = "crawling/naver_blog_reviews_removed_final.csv"
ROWLEVEL_OUT = "/sent_by_entity_rowlevel.csv"
SUMMARY_OUT  = "/sent_by_entity_summary.csv"

# ===== 모델 설정 =====
SENT_MODEL = "nlp04/korean_sentiment_analysis_kcelectra"  # 한국어 감성 분석 모델 예시
BATCH_SIZE = 32
CONTEXT_WINDOW = 1      # 매칭 문장 앞뒤로 포함할 문맥 문장 수
USE_EMBED_FILTER = False # True로 두면 임베딩 유사도 필터 사용(옵션)
SIM_MODEL = "BM-K/KoSimCSE-roberta-multitask"
SIM_THRESHOLD = 0.35

# ===== 고정 엔티티 목록 =====
ENTITIES = [
    "도산근린공원", "율현공원", "경의선숲길", "문화비축기지", "올림픽공원",
    "송파나루근린공원", "석촌호수", "인왕산도시자연공원", "인왕산", "낙산공원",
    "북한산국립공원", "북한산", "서울로 7017", "남산공원", "용산가족공원",
    "효창근린공원", "효창공원", "강서한강공원", "난지한강공원", "양화한강공원",
    "망원한강공원", "여의도한강공원", "이촌한강공원", "반포한강공원", "잠원한강공원",
    "뚝섬한강공원", "잠실한강공원", "광나루한강공원"
]



## 1) 클린업 전처리 (노이즈 제거용 토큰화)


In [2]:

EMOJI_PATTERN = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags
    "]+", flags=re.UNICODE)

URL_PATTERN = re.compile(r"https?://\S+|www\.\S+")
HTML_TAG = re.compile(r"<[^>]+>")
MULTISPACE = re.compile(r"\s{2,}")
REPEAT_CHAR = re.compile(r"(.)\1{2,}")  # 같은 문자 3회 이상 반복 → 2회로 축소

BLOG_NOISE = [
    "© NAVER Corp.", "이웃추가", "공유하기", "광고", "협찬", "체험단",
    "무단 전재", "리그램", "포스팅", "사진첨부"
]

def clean_text(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    t = text
    t = HTML_TAG.sub(" ", t)
    t = URL_PATTERN.sub(" ", t)
    t = EMOJI_PATTERN.sub(" ", t)
    for noise in BLOG_NOISE:
        t = t.replace(noise, " ")
    t = REPEAT_CHAR.sub(r"\1\1", t)   # ㅋㅋㅋㅋ → ㅋㅋ
    t = re.sub(r"[\r\t]+", " ", t)
    t = MULTISPACE.sub(" ", t).strip()
    return t



## 2) 문장 토큰화


In [3]:

try:
    import kss
    def sent_splitter(text: str):
        try:
            sents = kss.split_sentences(text)
            return [s.strip() for s in sents if s and s.strip()]
        except Exception:
            pass
except Exception:
    kss = None

import re as _re
def regex_sentence_split(text: str):
    t = _re.sub(r"[\r\t]+", " ", text)
    t = _re.sub(r"([.!?])", r"\1 ", t)
    sents = [s.strip() for s in _re.split(r"[\n]+|(?<=[.!?])\s+", t) if s.strip()]
    return sents

if 'sent_splitter' not in globals():
    def sent_splitter(text: str):
        return regex_sentence_split(text)



## 3) 엔티티 관련 문장 선별


In [4]:

def extract_relevant_sentences(text: str, entity: str, context_window: int = 1) -> List[str]:
    sents = sent_splitter(text)
    hits = set(i for i, s in enumerate(sents) if entity in s)
    if not hits:
        return []
    ctx_idx = set()
    for i in hits:
        for j in range(max(0, i - context_window), min(len(sents), i + context_window + 1)):
            ctx_idx.add(j)
    return [sents[i] for i in sorted(ctx_idx)]



### (선택) 임베딩 유사도 필터


In [5]:

EMBEDDER = None
if USE_EMBED_FILTER:
    from sentence_transformers import SentenceTransformer
    EMBEDDER = SentenceTransformer(SIM_MODEL)

def embed_filter(sentences: List[str], entity: str, threshold: float = 0.35) -> List[str]:
    if not sentences or EMBEDDER is None:
        return sentences
    prompt = f"{entity} 공원/산 관련 후기/시설/경관/산책/등산/접근성/혼잡도/청결/안전/만족도"
    q = EMBEDDER.encode([prompt], convert_to_tensor=True, normalize_embeddings=True)
    c = EMBEDDER.encode(sentences, convert_to_tensor=True, normalize_embeddings=True)
    sims = (q @ c.T).squeeze(0)  # cosine
    return [s for s, sim in zip(sentences, sims) if float(sim) >= threshold]



## 4) 데이터 로드 → 클린업 → 문장 선별


In [6]:

df = pd.read_csv(INPUT_PATH)

needed_cols = {"search_query", "content"}
missing = needed_cols - set(df.columns)
if missing:
    raise ValueError(f"입력 CSV에 필요한 컬럼이 없습니다: {missing}")

rows = []
for _, r in df.iterrows():
    content_raw = str(r["content"])
    content = clean_text(content_raw)
    search_q = str(r["search_query"])
    for entity in ENTITIES:
        if (entity in search_q) or (entity in content):
            sents = extract_relevant_sentences(content, entity, context_window=CONTEXT_WINDOW)
            sents = embed_filter(sents, entity, threshold=SIM_THRESHOLD) if USE_EMBED_FILTER else sents
            if sents:
                rows.append({
                    "search_query": search_q,
                    "entity": entity,
                    "relevant_text": " ".join(sents),
                    "content_clean": content
                })

df_ex = pd.DataFrame(rows)
print(f"선별된 행 수: {len(df_ex)}")
df_ex.head(3)


[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class

/Users/daeunpark/Documents/seoul_datahub_contest/babo/lib/python3.12/site-packages/pecab/_tokenizer.py:265: RuntimeWarning: overflow encountered in scalar add
  from_pos_data.costs[idx]
/Users/daeunpark/Documents/seoul_datahub_contest/babo/lib/python3.12/site-packages/pecab/_tokenizer.py:274: RuntimeWarning: overflow encountered in scalar add
  least_cost += word_cost


선별된 행 수: 514


,search_query,entity,relevant_text,content_clean
0,도산근린공원 후기,도산근린공원,도산근린공원 서울 도심의 숨겨진 명소 도산공원 나들이 복잡한 서울 도심 한복판 쾌적...,도산근린공원 서울 도심의 숨겨진 명소 도산공원 나들이 복잡한 서울 도심 한복판 쾌적...
1,도산근린공원 후기,도산근린공원,도산안창호선생기념사업회 도산 안창호 기념관 도산근린공원 ✔️ 관람장소 : 도산 안창...,도산안창호선생기념사업회 도산 안창호 기념관 도산근린공원 ✔️ 관람장소 : 도산 안창...
2,도산근린공원 후기,도산근린공원,나는 압구정 로데오역 5번 출구에서 걸어왔다. 공원의 사진들 도산근린공원 소개 이 ...,방문 계기 및 위치 50m 도산공원 서울특별시 강남구 도산대로45길 20 도산전시관...


In [8]:
# 선별 결과 CSV 저장
OUTPUT_PATH = "selected_sentences.csv"  # 원하는 파일명
df_ex.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"저장 완료: {OUTPUT_PATH} (총 {len(df_ex)}행)")


저장 완료: selected_sentences.csv (총 514행)



## 5) 딥러닝 감성분석 (Transformers)


In [7]:

from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
clf = pipeline("sentiment-analysis", model=SENT_MODEL, device=device)

def normalize_label(label: str) -> str:
    l = label.strip().lower()
    if l in ("positive", "긍정", "pos", "1"): return "positive"
    if l in ("negative", "부정", "neg", "0"): return "negative"
    return "negative"

def to_positive_prob(label: str, score: float) -> float:
    lab = normalize_label(label)
    return float(score) if lab == "positive" else float(1.0 - score)

pred_label, pred_score, pos_prob = [], [], []

texts = df_ex["relevant_text"].astype(str).tolist()
for i in range(0, len(texts), BATCH_SIZE):
    batch = texts[i:i+BATCH_SIZE]
    outs = clf(batch)
    for o in outs:
        lab = normalize_label(o["label"])
        sc  = float(o["score"])
        pred_label.append(lab)
        pred_score.append(sc)
        pos_prob.append(to_positive_prob(lab, sc))

df_ex["pred_label"] = pred_label
df_ex["pred_score"] = pred_score
df_ex["positive_prob"] = pos_prob

df_ex.head(5)


/Users/daeunpark/Documents/seoul_datahub_contest/babo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (529) must match the size of tensor b (512) at non-singleton dimension 1


## 6) 결과 저장 & 엔티티별 집계


In [ ]:

df_ex.to_csv(ROWLEVEL_OUT, index=False, encoding="utf-8-sig")

summary = (
    df_ex.groupby("entity", as_index=False)
         .agg(
             n_docs=("pred_label", "size"),
             n_positive=("pred_label", lambda s: (s == "positive").sum()),
             n_negative=("pred_label", lambda s: (s == "negative").sum()),
             positive_rate=("pred_label", lambda s: (s == "positive").mean()),
             mean_positive_prob=("positive_prob", "mean"),
         )
         .sort_values(["positive_rate", "mean_positive_prob", "n_docs"],
                      ascending=[False, False, False])
)
summary.to_csv(SUMMARY_OUT, index=False, encoding="utf-8-sig")

summary.head(10)



## 7) 시각화 (상위 20개 긍정 비율 기준)


In [ ]:

import matplotlib.pyplot as plt

top = summary.head(20)
plt.figure()
plt.bar(top["entity"], top["positive_rate"])
plt.xticks(rotation=45, ha="right")
plt.title("Top-20 Positive Rate by Entity")
plt.ylabel("Positive Rate")
plt.tight_layout()
plt.show()
